In [5]:
import re 
import pandas as pd
import os 
import html

In [19]:
os.chdir('/Users/lindsayduca/Desktop/Downloads')
#file="US20220000001A1-20220106.XML" 
#file="USD0864516-20191029.XML" 

file = open(file="ipa220106.txt", mode='r') #opening the file in read mode
file_content_raw = file.read()
file.close()
text1=re.compile("<\?xml version=\"1\.0\" encoding\=\"UTF\-8\"\?>")
file_content = text1.split(file_content_raw)
while '' in file_content:
    file_content.remove('')
print("No of patents :", len(file_content))    

No of patents : 7882


In [31]:
# Writing regular expressions for the desired columns

#grant_id = re.compile('file\=\"([U][S]\w\w\d{6})\-\d{8}\.XML\"')   
grant_id = re.compile('file\=\"([U][S]\w{13})\-\w{8}\.[X][M][L]\"')   

#patent_title = re.compile("<invention-title id=\"\w{5,6}\">(.*?)</invention-title>") 
kind = re.compile("<kind>([A-Z]\d)</kind>")
#number_of_claim = re.compile("\<number\-of\-claims\>(\d{1,4})\<\/number\-of\-claims\>")
first_name=re.compile("<first-name>(.*?)</first-name>")
last_name=re.compile("<last-name>(.*?)</last-name>")
citation_by_examiner = re.compile("\<category\>cited by examiner<\/category\>")
citation_by_applicant = re.compile("\<category>cited by applicant\<\/category\>")
claim_text=re.compile("<claim-text>[\s\S<]*</claim-text>")
abstract=re.compile("\<abstract id\=\"abstract\"\>\n\<p id\=\"p\-0001\" num\=\"0000\"\>(.*?)\<\/p\>\n\<\/abstract\>")

# some extra care with claim-text
cleaner = re.compile('<.*?>') 
cleaner2 = re.compile('\n')
cleaner3 = re.compile('\,\,\,')
cleaner4 = re.compile("[\.][\,][\,]")
cleaner5 = re.compile("[\,][\,]")
cleaner6 = re.compile("[\;][\,]")

In [26]:
for line in file_content:
    
    gid=grant_id.findall(line) #to find grant_id
    
# checking length of gid is not equal to 0 then do append to all the lists
    if len(gid)!=0:                             
        gid_list.append(gid[0])
        
data_frame = pd.DataFrame(
    {'grant_id': gid_list})
data_frame

grant_id
0      US20220000001A1
1      US20220000001A1
2      US20220000002A1
3      US20220000003A1
4      US20220000004A1
...                ...
22673  US20220007555A1
22674  US20220007556A1
22675  US20220007557A1
22676  US20220007558A1
22677  US20220007559A1

[22678 rows x 1 columns]

In [ ]:
gid_list, title_list, kind_list, no_of_claim_list, name_list, applicant_list, examiners_list, claim_list, abstract_list, = ([] for i in range(9))

for line in file_content:
    
    gid=grant_id.findall(line) #to find grant_id
    title=patent_title.findall(line) #to find patent_title
    kinds=kind.findall(line) #to find kind
   # sclaim=number_of_claim.findall(line) #to find no_of_claims
     #to find inventors
    inventors=re.findall("<inventor.*?>[\s\S]*</inventor>",line)
    for person in inventors:
        first=first_name.findall(person)
        last=last_name.findall(person)
    name = [firstName +" "+ lastName for firstName, lastName in zip(first,last)]
    if len(name)==0:
        names="NA"
    else:
        names=name
    
    # here we count citation_by_applicant
    if len(citation_by_applicant.findall(line))==0:
        citation_by_applicants=0
    else:
        citation_by_applicants=len(citation_by_applicant.findall(line)) 
    
    # count for citation_by_examiner
    if len(citation_by_examiner.findall(line))==0:
        citation_by_examiners=0
    else:    
        citation_by_examiners=len(citation_by_examiner.findall(line))   
    
    # For claim_text
    if (len(re.findall("<claim-text>[\s\S<]*</claim-text>",line))==0):
        claim_text=["NA"]
    else:
        claim_text=re.findall("<claim-text>[\s\S<]*</claim-text>",line) 
    
    # For abstract
    abst=abstract.findall(line)
    if len(abst)==0:
        abstracts=["NA"]
    else:    
        abstracts=abst  
    
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(gid)!=0:                             
        gid_list.append(gid[0])
       # title_list.append(title[0])
        kind_list.append(kinds[0])
      #  no_of_claim_list.append(sclaim[0])
        name_list.append(names)
        applicant_list.append(citation_by_applicants)
        examiners_list.append(citation_by_examiners)
        claim_list.append(claim_text[0])
        abstract_list.append(abstracts[0])
"""     
# cleaning claim text         
element=0
for items in claim_list:
    claim_list[element]=re.sub(cleaner,'',claim_list[element])
    claim_list[element]=re.sub(cleaner2,',',claim_list[element])
    claim_list[element]=re.sub(cleaner3,',',claim_list[element])
    claim_list[element]=re.sub(cleaner4,'.,',claim_list[element])
    claim_list[element]=re.sub(cleaner5,',',claim_list[element])
    claim_list[element]=re.sub(cleaner6,'; ',claim_list[element])
    element=element+1
"""
# For kind 
Kind1 = [w.replace('P2', 'Plant Patent Grant(with a published application) issued on or after January 2, 2001') for w in kind_list]
Kind2 = [w.replace('B2', 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.') for w in Kind1]
Kind3 = [w.replace('S1', 'Design Patent') for w in Kind2]
Kind4 = [w.replace('B1', 'Utility Patent Grant (no published application) issued on or after January 2, 2001.') for w in Kind3]

In [ ]:
# Creating data frame
data_frame = pd.DataFrame(
    {'grant_id': gid_list,
     #'patent_title': title_list,
     'kind': Kind4,
     #'number_of_claims':no_of_claim_list,
     'inventors':name_list,
     'citations_applicant_count':applicant_list,
     'citations_examiner_count':examiners_list,
     'claims_text':claim_list,
     'abstract':abstract_list
    })

In [ ]:
data_frame.isnull().sum()

In [ ]:
data_frame

In [ ]:
data_frame.to_csv('ParsedPatentGrant.csv')

In [38]:
import requests
from bs4 import BeautifulSoup
import json

r = requests.get("https://developer.uspto.gov/ibd-api/v1/patent/application?patentNumber=9876543&start=0&rows=100")
soup = BeautifulSoup(r.text, "html.parser")
text = soup.get_text()

r_dict = json.loads(str(text))
print(r_dict['response']['docs'][0]['inventor'][0])

SSLError: HTTPSConnectionPool(host='developer.uspto.gov', port=443): Max retries exceeded with url: /ibd-api/v1/patent/application?patentNumber=9876543&start=0&rows=100 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))